In [1]:
# %load_ext autoreload
# %autoreload 2

# import importlib
from stylegan2.solver import Solver

import numpy as np

In [2]:
import random
from torch.utils.data.sampler import WeightedRandomSampler

def make_balanced_sampler(labels):
    class_counts = np.bincount(labels)
    class_weights = 1. / class_counts
    weights = class_weights[labels]
    WeightedRandomSampler(weights, len(weights))

In [3]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

img_size = 128
batch_size = 4
num_workers = 1

transform = transforms.Compose([
    transforms.Resize([img_size, img_size]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5]),
])

ds = ImageFolder('./data/mai', transform)

sampler = make_balanced_sampler(ds.targets)
loader = DataLoader(dataset=ds,
                    batch_size=batch_size,
                    sampler=sampler,
                    shuffle=True,
                    num_workers=num_workers,
                    pin_memory=True,
                    drop_last=True)

# val_dataset = ImageFolder('/home/aray/stargan-v2/data/afhq/val', transform)


domains, domains_mapping = ds._find_classes(ds.root)
n_domains = len(domains)

print('n_domains:', n_domains)
domains_mapping

n_domains: 2


{'wsi1_tiles': 0, 'wsi2_tiles': 1}

In [4]:
solver = Solver('StyleGAN_mai_128', '0', img_size)

In [ ]:
solver.train(loader, batch_size)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: array (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.19 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


In [6]:
solver.save_model()